In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

In [3]:
# 確認是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Data Preprocessing

In [4]:
# TODO: 針對train set 進行數據增強和正規化
train_transform = transforms.Compose([ # 將所有轉換操作依次應用到每張圖片
    transforms.Resize((64, 64)), # 調整圖片大小
    transforms.RandomHorizontalFlip(), # 隨機水平翻轉圖片-> 增強資料多樣性，模擬可能出現的視覺變化
    transforms.RandomRotation(20), # 將圖片隨機旋轉 -> 增加資料的旋轉不變性
    transforms.RandomResizedCrop(64), # 機裁剪圖片的某個區域，並調整到64x64
    transforms.ToTensor(), # PIL圖片格式轉換為tensor
    # 資料[0, 255]正規化為[0.0, 1.0]
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5], 
        std=[0.5, 0.5, 0.5]
        ) 
])

In [5]:
# TODO: 針對test set 進行正規化
test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

## 載入資料集

In [8]:
train_dataset = datasets.ImageFolder("D:/Downloads/dataset/training_set", transform=train_transform)
test_dataset = datasets.ImageFolder("D:/Downloads/dataset/test_set", transform=test_transform)

In [9]:
# 定義資料載入器
# shuffle 控制是否隨機打亂數據集
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
print(f"類別: {train_dataset.classes}")

類別: ['cats', 'dogs']


## 建立 CNN

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [12]:
model = CNN().to(device) # 建立模型

## 訓練 CNN

In [13]:
criterion = nn.BCELoss()  # 二元交叉熵損失函數
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# 訓練循環
epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 前向傳播
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())

        # 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # 計算精確度
        predictions = (outputs.squeeze() > 0.5).float()  # 二元分類（假設輸出是概率）
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)
    
    # 計算平均損失和精確度
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

Epoch 1/25, Loss: 0.6789, Accuracy: 0.5647
Epoch 2/25, Loss: 0.6457, Accuracy: 0.6278
Epoch 3/25, Loss: 0.6316, Accuracy: 0.6400
Epoch 4/25, Loss: 0.6216, Accuracy: 0.6418
Epoch 5/25, Loss: 0.6134, Accuracy: 0.6616
Epoch 6/25, Loss: 0.6117, Accuracy: 0.6573
Epoch 7/25, Loss: 0.6069, Accuracy: 0.6679
Epoch 8/25, Loss: 0.6012, Accuracy: 0.6729
Epoch 9/25, Loss: 0.5940, Accuracy: 0.6760
Epoch 10/25, Loss: 0.5938, Accuracy: 0.6734
Epoch 11/25, Loss: 0.5937, Accuracy: 0.6833
Epoch 12/25, Loss: 0.5897, Accuracy: 0.6786
Epoch 13/25, Loss: 0.5952, Accuracy: 0.6736
Epoch 14/25, Loss: 0.5865, Accuracy: 0.6807
Epoch 15/25, Loss: 0.5841, Accuracy: 0.6874
Epoch 16/25, Loss: 0.5817, Accuracy: 0.6875
Epoch 17/25, Loss: 0.5771, Accuracy: 0.6916
Epoch 18/25, Loss: 0.5825, Accuracy: 0.6915
Epoch 19/25, Loss: 0.5747, Accuracy: 0.6957
Epoch 20/25, Loss: 0.5769, Accuracy: 0.6930
Epoch 21/25, Loss: 0.5733, Accuracy: 0.6997
Epoch 22/25, Loss: 0.5662, Accuracy: 0.7014
Epoch 23/25, Loss: 0.5657, Accuracy: 0.70